In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
## Langchain Tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "True"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [3]:
## Data Ingension from  the website we need to scrape
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://bek-next-portfolio.vercel.app/about")
data = loader.load()
print(data)

d:\Python\Python-Programming\Complete_Agentic_AI_Bootcamp_with_LangGraph_and_LangChain\Section_6_Open_AI_and_Ollama\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://bek-next-portfolio.vercel.app/about', 'title': 'About | Otabek', 'description': 'John Doe is a developer, writer and speaker. He is a digital nomad and travels around the world while working remotely.', 'language': 'en'}, page_content="About | Otabek OtabekDeveloperHomeAboutProjectsArticlesChallengesContactSocialsLinkedInGithubLeetCodeInstagramEnglish한국어O'zbek tili💬About MeSelf IntroductionHello!My name is Otabek, and I am a passionate junior developer who loves new challenges and learning.\n          I am from Uzbekistan and have been living in South Korea for six and a half years. \n          I graduated from the Computer Engineering department at Seoul National University of Science and Technology in February 2025.\n          Since living in Korea for over six years, I have become well-accustomed to Korean culture and can communicate fluently in Korean.\n          I can also handle basic daily conversations in English and am continuously studyi

In [4]:
### Load Data->Docs->Divide into Chunks, ->Embeddings ->Store in Vector DB
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents=text_splitter.split_documents(data)
for doc in documents:
    print(doc.page_content)




About | Otabek OtabekDeveloperHomeAboutProjectsArticlesChallengesContactSocialsLinkedInGithubLeetCodeInstagramEnglish한국어O'zbek tili💬About MeSelf IntroductionHello!My name is Otabek, and I am a passionate junior developer who loves new challenges and learning.
          I am from Uzbekistan and have been living in South Korea for six and a half years. 
          I graduated from the Computer Engineering department at Seoul National University of Science and Technology in February 2025.
          Since living in Korea for over six years, I have become well-accustomed to Korean culture and can communicate fluently in Korean.
          I can also handle basic daily conversations in English and am continuously studying to improve my fluency for smoother communication in the future.From a young age, I have been deeply interested in digital devices, computers, games, and programming,
which led me to the field of development. I particularly enjoy thinking creatively and creating unique and spe

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings=OpenAIEmbeddings()
vectorStoreDB=FAISS.from_documents(documents,embeddings)

In [7]:
query="I have been working on various projects utilizing technologies like Node.js"
result=vectorStoreDB.similarity_search(query)
content=result[0].page_content
print(len(content))

887


In [8]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x0000027F4E261210> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000027F4E263160> root_client=<openai.OpenAI object at 0x0000027F4E260CD0> root_async_client=<openai.AsyncOpenAI object at 0x0000027F4E2628F0> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********') stream_usage=True


In [14]:
## Retrival Chain and Document Chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

prompt=ChatPromptTemplate.from_template(
    """
    Anwere the following question based on the context provided.
    <context>
    {context}
    </context>
    """
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain



RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Anwere the following question based on the context provided.\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000027F4E261210>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000027F4E263160>, root_client=<openai.OpenAI object at 0x0000027F4E260CD0>, root_async_client=<openai.AsyncOpenAI object at 0x0000027F4E2628F0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)
| StrOu

In [17]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":" Recently, What I have been working on?",
   "context":[Document(page_content="I have been working on various projects utilizing technologies like Node.js, Express.js, MongoDB, and MySQL, which has helped me become proficient in backend development as well. Recently, I have been focusing on full-stack development using server-side rendering (SSR) with Next.js and TypeScript, aiming to grow into a full-stack developer. Of course, my backend experience is relatively limited, but I am continuously learning and improving my skills to bridge the gap.")]
})



'What specific technologies has the person been focusing on in their journey to become a full-stack developer?'

In [19]:
### Retriever is a tool that can be used to retrieve documents from the vector store.
retriever=vectorStoreDB.as_retriever()
from langchain_classic.chains import create_retrieval_chain

retrieval_chain=create_retrieval_chain(retriever,document_chain)
print(retrieval_chain)






bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000027F4E2606D0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Anwere the following question based on the context provided.\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
            | ChatO

In [20]:
## Get the response from the LLM
response=retrieval_chain.invoke({"input":" Recently, What I have been working on?"})
print(response["answer"])

What technologies and concepts has Otabek gained proficiency in?

Based on the context provided, Otabek has gained proficiency in several technologies and concepts:

1. **Front-End Technologies**: 
   - JavaScript, HTML, and CSS.
   - Modern UI design tools such as Tailwind CSS, SASS, and Styled-Components.
   - React, including component-based UI development, one-way data flow, and efficient Virtual DOM rendering.
   - Essential Hooks and state management in React.
   - CSS techniques, responsive design, and implementation (both Mobile First and Desktop First approaches).

2. **Back-End Technologies**:
   - Node.js, Express.js, MongoDB, and MySQL.

3. **Full-Stack Development**:
   - Focus on using server-side rendering (SSR) with Next.js and TypeScript.

In addition to technical skills, Otabek has a passion for learning, enjoys creative thinking, and is committed to becoming a competent developer who excels both technically and in teamwork.
